# Lasso Example

In [38]:
import PyPlot
import ProximalOPT
import HD

In [178]:
reload("ProximalOPT")

Generate Data

In [114]:
srand(1)

n = 500
p = 2500

t_beta = sprandn(p,1,0.05) 
t_beta = vec(full(t_beta))
A = randn(n,p)
A = A ./ mapslices(norm, A, 1)
v = sqrt(0.001)*randn(n)
b = A*t_beta + v

println("solving instance with $n examples, $p variables")
println("nnz(x0) = $(countnz(t_beta)); signal-to-noise ratio: $(norm(A*t_beta)^2/norm(v)^2)")

AA = A'A 
Ab = A'b 

gamma_max = norm(A'*b, Inf)
gamma = 0.1*gamma_max;

solving instance with 500 examples, 2500 variables
nnz(x0) = 129; signal-to-noise ratio: 251.22786125240205


Proximal gradient

In [176]:
h_beta = zeros(p)

tic()
f = ProximalOPT.quad_f(AA, -Ab)
g = ProximalOPT.prox_l1(gamma)
tr = ProximalOPT.prox_grad!(h_beta, f, g; beta=0.9, show_trace = false, ABSTOL=1e-4)
prox_res = (copy(h_beta), tr, toc());

elapsed time: 1.812498264 seconds


Fast proximal gradient

In [180]:
h_beta1 = zeros(p)

tic()
f = ProximalOPT.quad_f(AA, -Ab)
g = ProximalOPT.prox_l1(gamma)
tr = ProximalOPT.acc_prox_grad!(h_beta1, f, g; beta=0.9, show_trace = false, ABSTOL=1e-4)
acc_prox_res = (copy(h_beta1), tr, toc());

elapsed time: 1.508100274 seconds


In [182]:
h_beta2 = spzeros(p,1)
lambda = gamma * ones(p)
HD.lasso!(h_beta2, AA, Ab, lambda)
h_beta2 = vec(full(h_beta2));

In [183]:
maximum(abs(h_beta1 - h_beta2))

0.0177064803193141

In [186]:
obj_val(x) = dot(x, AA*x) / 2. - dot(x, Ab) + gamma * norm(x, 1) + dot(b, b) / 2.

obj_val (generic function with 1 method)

In [187]:
obj_val(h_beta), obj_val(h_beta1), obj_val(h_beta2)

(26.295687940948334,26.29633781251576,26.295685167540775)